In [4]:
!pip3 install pandas

    100% |████████████████████████████████| 14.4MB 1.1MB/s eta 0:00:01   44% |██████████████▎                 | 6.4MB 16.8MB/s eta 0:00:01
    100% |████████████████████████████████| 24.5MB 514kB/s ta 0:00:011   29% |█████████▍                      | 7.2MB 5.0MB/s eta 0:00:04    87% |███████████████████████████▉    | 21.3MB 14.3MB/s eta 0:00:01    95% |██████████████████████████████▍ | 23.3MB 19.8MB/s eta 0:00:01


In [5]:
!pip3 install sklearn

    100% |████████████████████████████████| 7.8MB 3.1MB/s eta 0:00:01  5% |█▋                              | 399kB 4.3MB/s eta 0:00:02
    100% |████████████████████████████████| 16.7MB 1.3MB/s eta 0:00:01   15% |█████                           | 2.6MB 14.0MB/s eta 0:00:02    60% |███████████████████▍            | 10.1MB 8.9MB/s eta 0:00:01    91% |█████████████████████████████▎  | 15.3MB 17.1MB/s eta 0:00:01
  Running setup.py bdist_wheel for sklearn ... done
  Stored in directory: /Users/emil/Library/Caches/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [6]:
!pip3 install xgboost

    100% |████████████████████████████████| 645kB 6.9MB/s ta 0:00:01
  Running setup.py bdist_wheel for xgboost ... done
  Stored in directory: /Users/emil/Library/Caches/pip/wheels/8a/89/0c/4870bb69132698f40889fa956f92e630a36c1fd7173fcc759f
Successfully built xgboost


In [8]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
import sklearn.preprocessing
from xgboost import XGBClassifier

In [17]:
train_file = "../data/train.csv"
test_file = "../data/test.csv"


train_data_raw = pd.read_csv(train_file)
test_data_raw = pd.read_csv(test_file)

target = "Survived"


In [18]:

### CLEAN DATA FUNC

def clean_func(train_data):
    
    ## DO IMPUTATION 
    # FARE
    imp_fare = Imputer(missing_values="NaN", strategy="mean")
    imp_fare.fit(train_data[["Fare"]])
    train_data[["Fare"]]=imp_fare.transform(train_data[["Fare"]]).ravel() 

    # Age
    imp=Imputer(missing_values="NaN", strategy="mean")
    imp.fit(train_data[["Age"]])
    train_data[["Age"]]=imp.transform(train_data[["Age"]]).ravel() 
    
    # Filna
    train_data["Cabin"] = train_data["Cabin"].fillna("")

    
    # one hot encoding
    sex_features = pd.get_dummies(train_data["Sex"])
    embarked_features = pd.get_dummies(train_data["Embarked"])
    
    # rename embarked features
    embarked_features = embarked_features.rename(columns={'C': 'embarked_cobh'
                                                        , 'Q': 'embark_queenstown'
                                                        , 'S': 'embark_southampton'})

    # Concat new features
    train_data_extras = pd.concat([train_data,sex_features,embarked_features],axis=1)

    
    
    # HACK - REMOVE T WHICH IS NOT IN TEST LIKELY ERRROR 
    cabin_letters = pd.get_dummies(train_data['Cabin'].map(lambda x: "empty" if len(x)==0 or x[0]=="T" else x[0]))

#    cabin_letters = pd.get_dummies(train_data['Cabin'].map(lambda x: "empty" if len(x)==0 else x[0]))
    cabin_letters.columns = ["Cabin_letter_"+i for i in cabin_letters.columns]
    train_data_extras = pd.concat([train_data_extras,cabin_letters],axis=1)
    

    train_data_extras["Cabin_number"] = train_data['Cabin'].map(lambda x: -99 if len(x)==0 else x.split(" ")[0][1:]) 

    # ONLY RETURN NUMERIC COLUMNS 
    num_types = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64','uint8']
    train_data_numerics = train_data_extras.select_dtypes(include=num_types)

    return train_data_numerics


## Select only numeric columns

In [19]:
train_data_raw2 = clean_func(train_data_raw)
train_data = train_data_raw2.iloc[:, train_data_raw2.columns != target]
train_data_target = train_data_raw2[target].values


/usr/local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [20]:
X_train,X_test,Y_train,Y_test = train_test_split(train_data
                              ,train_data_target
                              ,test_size=0.3
                              ,random_state=42)

# Models
- logreg
- random forest

In [22]:
log_reg = LogisticRegression(penalty="l2", dual=False, tol=0.0001, C=1.0
                             , fit_intercept=True, intercept_scaling=1
                             , class_weight=None, random_state=None
                             , solver="liblinear", max_iter=100
                             , multi_class="ovr", verbose=0
                             , warm_start=False, n_jobs=1)

log_reg.fit(X_train,Y_train)




LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [23]:
Y_pred = log_reg.predict(X_test)

In [24]:
metrics.accuracy_score(Y_test,Y_pred) 

0.8022388059701493

### random forest naive

In [25]:
model_rf = RandomForestClassifier(
n_estimators=100
)

model_rf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [26]:
# Cross Validation RF

scores = cross_val_score(model_rf, X_train, Y_train, cv=10)
print(scores)

[0.8125     0.79365079 0.88709677 0.85483871 0.82258065 0.75806452
 0.82258065 0.77419355 0.82258065 0.93548387]


In [27]:
pred_rf = model_rf.predict(X_test)
metrics.accuracy_score(Y_test,pred_rf)

0.8097014925373134

### Random Forest Grid Search

In [28]:
model_rf_gs = RandomForestClassifier()



In [29]:
# parmeter dict
param_grid = dict(
    n_estimators=np.arange(60,101,20)
    , min_samples_leaf=np.arange(2,4,1)
    #, criterion = ["gini","entropy"]
    #, max_features = np.arange(0.1,0.5,0.1)
)
print(param_grid)

{'n_estimators': array([ 60,  80, 100]), 'min_samples_leaf': array([2, 3])}


In [30]:
grid = GridSearchCV(model_rf_gs,param_grid=param_grid,scoring = "accuracy", cv = 5)
grid.fit(train_data, train_data_target)
""

# model_rf.fit(train_data, train_data[target])

''

In [31]:
#print(grid)
# for i in ['params',"mean_train_score","mean_test_score"]:
#     print(i)
#     print(grid.cv_results_[i])
#grid.cv_results_

In [32]:
print(grid.best_params_)
print(grid.best_score_)


{'min_samples_leaf': 2, 'n_estimators': 60}
0.8260381593714927


### XGBOOST

In [33]:
model_xgboost = XGBClassifier(max_depth=7, learning_rate=0.01, n_estimators=20)

In [34]:
model_xgboost.fit(X_train, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=20,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [35]:
print(metrics.accuracy_score(Y_train,model_xgboost.predict(X_train)))
metrics.accuracy_score(Y_test,model_xgboost.predict(X_test))


0.8860353130016051


0.8246268656716418

## train on full data - can only use estimate from cross val

In [36]:
model_rf_gs = RandomForestClassifier(**grid.best_params_)
model_rf_gs.fit(train_data,train_data_target)
""
#print(**grid.best_params_)



''

### Get params - lin reg

In [38]:
# get parameters
coef = list(log_reg.coef_.ravel())
intercept = log_reg.intercept_

# print them
print(intercept)

coef
for id, i in enumerate(coef):
    print(train_data.columns[id],i)


[1.2621408]
PassengerId 0.00029754297439010076
Pclass -0.6157041035123048
Age -0.02903072567345783
SibSp -0.25732374714333606
Parch -0.09867922601557672
Fare 0.004641851387090121
female 1.9122612386721238
male -0.6501204412377364
embarked_cobh 0.7087649387386186
embark_queenstown 0.40782169067824925
embark_southampton 0.07640570396422271
Cabin_letter_A -0.11839648944772779
Cabin_letter_B 0.2569960518896235
Cabin_letter_C -0.568454858657011
Cabin_letter_D 0.5681776752344649
Cabin_letter_E 1.2341622670757595
Cabin_letter_F 0.7593246563167335
Cabin_letter_G -0.5404146082007102
Cabin_letter_empty -0.3292538967767734


# PREDICT AND STORE OUTPUT

In [39]:
### HACK TO COMPUTE TEST RESULT
test_data = clean_func(test_data_raw)

#test_data[["Age"]]=imp.transform(test_data[["Age"]]).ravel()

/usr/local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [40]:
## DO IMPUTATION ON FARE
# imp_fare = Imputer(missing_values="NaN", strategy="mean")
# imp_fare.fit(train_data[["Fare"]])

# test_data[["Fare"]]=imp_fare.transform(test_data[["Fare"]]).ravel() # what is ravel???


In [41]:
test_data.isnull().sum()


PassengerId           0
Pclass                0
Age                   0
SibSp                 0
Parch                 0
Fare                  0
female                0
male                  0
embarked_cobh         0
embark_queenstown     0
embark_southampton    0
Cabin_letter_A        0
Cabin_letter_B        0
Cabin_letter_C        0
Cabin_letter_D        0
Cabin_letter_E        0
Cabin_letter_F        0
Cabin_letter_G        0
Cabin_letter_empty    0
dtype: int64

In [42]:
#test_data_y = log_reg.predict(test_data)
test_data_y = log_reg.predict(test_data)


#train_data.head(3)

In [49]:
output = pd.DataFrame(list(zip(list(test_data["PassengerId"]),list(test_data_y))))

output.columns = ["PassengerId","Survived"]

In [50]:
output.to_csv(index=False, path_or_buf= "../data/output.csv")


# output of random forest



In [54]:
def output(data,file_name):
    output = pd.DataFrame(list(zip(list(test_data["PassengerId"]),list(data))))
    output.columns = ["PassengerId","Survived"]
    output.to_csv(index=False, path_or_buf= "../data/{file_name}.csv".format(file_name=file_name))

    


In [55]:
a =list(test_data.columns)
b = list(train_data.columns)

[item for item in b if item not in a]



[]

In [56]:
model_rf_data_y = model_rf.predict(test_data)

output(model_rf_data_y,"predict_rf_1")


In [57]:
model_rf_gs_data_y =model_rf_gs.predict(test_data)
output(model_rf_gs_data_y,"predict_rf_gs_1")

In [58]:
model_xgboost_data_y = model_xgboost.predict(test_data)
output(model_xgboost_data_y,"predict_xgboost_1")
